In [2]:
import pandas as pd
import numpy as np
import re
import os

def anger(l):
    x = [w for w in re.findall(r'\bliar\b|\bliars\b|\blying\b|\blies\b|\bhypocrite\b|\bhypocrites\b|\bhypocrisy\b|\bhypocritical\b|\basshole\b|\bassholes\b|\bbullshit\b|\bdisgrace\b|\bdisgraces\b|\bdisgraced\b|\bdisgraceful\b|\bstfu\b|\bdisgusting\b|\bdisgusted\b|\bdisgusts\b|\bscum\b|\binfuriate\b|\binfuriates\b|\binfuriating',l)]
    a = [w for w in re.findall(r'\bfuck you\b|\bthe fuck up\b|\bpiece of shit\b|\bgo fuck yourself\b',l)]
    b = [w for w in re.findall(r'\b(piss)\b.*off',l)]
    c = [w for w in re.findall(r'\boff\b.*\b(piss)\b',l)]
    d = [w for w in re.findall(r'\b(fuck)\b.*\boff\b',l)]
    e = [w for w in re.findall(r'\boff\b.*\b(fuck)\b',l)]
    z = []
    lists = [x,a,b,c,d,e,]
    for q in lists:
        for s in q:
            z.append(s)
    if len(z) > 0:
        return True
    else:
        return False
    
def day(l):
    a = str(l)
    return a[:10]

#Count angry tweets and total tweets per day in any Twitter data
def anger_time(filepath):
    df = pd.read_excel(filepath)
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False].copy()
    df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df = df[df['TEXT'].str.contains('FALSE')==False]
    df = df[df['TEXT'].str.contains('TRUE')==False].copy()
    df['Anger'] = df['TEXT'].str.lower().apply(anger)
    df['Day'] = df['CREATED_AT_LOCAL'].apply(day)
    dfangry = df[df['Anger']==True]
    days = df.pivot_table(index='Day',values='TEXT',aggfunc="count")
    daysa = dfangry.pivot_table(index='Day',values='TEXT',aggfunc="count")
    byday = days.join(daysa, how='left',lsuffix='_total',rsuffix='_angry')
    byday['Ratio'] = byday['TEXT_angry'] / byday['TEXT_total']
    byday['Text %'] = byday['TEXT_total'] / byday['TEXT_total'].sum()
    byday['Angry %'] = byday['TEXT_angry'] / byday['TEXT_angry'].sum()
    name = os.path.basename(filepath).split('.')[0] + ' anger by day.csv'
    byday.fillna(0).to_csv(os.path.join(os.path.dirname(filepath),name))

In [ ]:
anger_time('INSERT FILEPATH')